In [1]:
println("Is Scala working ?")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.85:4040
SparkContext available as 'sc' (version = 3.5.3, master = local[*], app id = local-1728994369382)
SparkSession available as 'spark'


Hello


In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [3]:
val spark = SparkSession.builder()
    .appName("AcademicGraphAnalysis")
    .getOrCreate()
spark.conf.set("spark.sql.caseSensitive", true)
import spark.implicits._

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2e05550e
import spark.implicits._


In [6]:
val jsonFilePath = "id2221/data/dblp_v14-part2266-3.json"

val papersDF = spark.read.option("multiline", "true")
    .json(jsonFilePath)
    .select(
        "id",
        "title",
        "doi",
        "keywords",
        "n_citation",
        "year",
        "issn",
        "url",
        "abstract",
        "authors",
        "doc_type",
        "v12_authors",
        "references",
        "v12_id"
    )

// View the schema
papersDF.printSchema()

// Show a few rows of the DataFrame
papersDF.show(1, truncate = false)

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n_citation: long (nullable = true)
 |-- year: long (nullable = true)
 |-- issn: string (nullable = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |-- doc_type: string (nullable = true)
 |-- v12_authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = tru

jsonFilePath: String = id2221/data/dblp_v14-part2266-3.json
papersDF: org.apache.spark.sql.DataFrame = [id: string, title: string ... 12 more fields]


In [7]:
val vertices: RDD[(VertexId, (String, String, Long))] = papersDF
    .select("id", "title", "year")
    .rdd
    .map(row => (row.getAs[String]("id").hashCode.toLong, (row.getAs[String]("id"), row.getAs[String]("title"), row.getAs[Long]("year"))))

vertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String, Long))] = MapPartitionsRDD[12] at map at <console>:39


In [8]:
val edges: RDD[Edge[String]] = papersDF
    .select("id", "references")
    .rdd
    .flatMap(row => {
        val paperId = row.getAs[String]("id").hashCode.toLong
        val references = row.getAs[Seq[String]]("references")
        
        // Check if references is not null before mapping
        if (references != null) {
            references.map(ref => Edge(paperId, ref.hashCode.toLong, "cites"))
        } else {
            // If references is null, return an empty list
            Seq.empty[Edge[String]]
        }
    })

edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = MapPartitionsRDD[18] at flatMap at <console>:39


In [9]:
val graph: Graph[(String, String, Long), String] = Graph(vertices, edges)

graph: org.apache.spark.graphx.Graph[(String, String, Long),String] = org.apache.spark.graphx.impl.GraphImpl@4b5d0c5c


In [10]:
val connectedComponents = graph.connectedComponents().vertices

// Join the connected component ID with the paper titles for interpretation
val connectedComponentsWithTitles = connectedComponents.join(vertices).map {
    case (id, (componentId, (paperId, title, year))) => (componentId, paperId, title, year)
}

connectedComponentsWithTitles.take(10).foreach(println)

(-1489633663,558b11bde4b031bae1fb5972,Close target reconnaissance using autonomous UAV formations,2008)


connectedComponents: org.apache.spark.graphx.VertexRDD[org.apache.spark.graphx.VertexId] = VertexRDDImpl[68] at RDD at VertexRDD.scala:57
connectedComponentsWithTitles: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String, String, Long)] = MapPartitionsRDD[86] at map at <console>:40


In [12]:
import scala.util.Random

// Step 1: Collect valid vertex IDs into a Set
val validVertexIds: Set[VertexId] = vertices.map(_._1).collect().toSet

// Step 2: Filter edges to include only those that point to valid vertices
val filteredEdges = edges.filter { edge =>
    validVertexIds.contains(edge.srcId) && validVertexIds.contains(edge.dstId)
}

// Step 3: Convert vertices to Sigma.js format
val random = new Random()
val nodesJSON = vertices.map { case (id, (paperId, title, year)) =>
  // Generate random coordinates (you may want to adjust the range as needed)
  val x = random.nextDouble() * 800  // Assuming width of SVG is 800
  val y = random.nextDouble() * 600  // Assuming height of SVG is 600
  JObject(
    "id" -> JInt(id), // Use paperId as the id for the node
    "label" -> JString(title),
    "x" -> JDouble(x),  // Use the generated x coordinate
    "y" -> JDouble(y),  // Use the generated y coordinate
    "size" -> JInt(3)   // Size can be arbitrary; adjust as necessary
  )
}.collect().toList

// Step 4: Convert the filtered edges RDD to a JSON array (JArray)
val edgesJSON = filteredEdges.zipWithIndex.map { case (Edge(srcId, dstId, relationship), index) =>
  JObject(
    "id" -> JString("e" + index), // Create a unique id for each edge
    "source" -> JInt(srcId),
    "target" -> JInt(dstId)
  )
}.collect().toList

// Step 5: Combine nodes and edges into a single JSON object for Sigma
val graphJSON: JObject = JObject(
  "nodes" -> JArray(nodesJSON),
  "edges" -> JArray(edgesJSON)
)

// Step 6: Convert to compact JSON string
val jsonString = compact(render(graphJSON))

// Step 7: Write the JSON to a file for Sigma.js
Files.write(Paths.get("./graph2.json"), jsonString.getBytes)


import scala.util.Random
validVertexIds: Set[org.apache.spark.graphx.VertexId] = Set(913341741)
filteredEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = MapPartitionsRDD[93] at filter at <console>:60
random: scala.util.Random = scala.util.Random@7ad73064
nodesJSON: List[org.json4s.JsonAST.JObject] = List(JObject(List((id,JInt(913341741)), (label,JString(Close target reconnaissance using autonomous UAV formations)), (x,JDouble(91.52470611315718)), (y,JDouble(73.11589797952477)), (size,JInt(3)))))
edgesJSON: List[org.json4s.JsonAST.JObject] = List()
graphJSON: org.json4s.JsonAST.JObject = JObject(List((nodes,JArray(List(JObject(List((id,JInt(913341741)), (label,JString(Close target reconnaissance using autonomous UAV formations)), (x,JDouble(91.52470611315718)), (y,...
